In [1]:
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
from datetime import date
from datetime import timedelta

In [2]:
def get_news_text(url):
    page_text = requests.get(url).text
    bs=BeautifulSoup(page_text, "html5lib")
    title = None
    # Заголовок
    title = bs.h1.text
    # Сам текст
    whole_text = bs.find_all('div', 'story__text')
    text = None
    for i in range(len(whole_text)):
        text = " ".join([p.text for p in whole_text[i].find_all("p")])
        if not text:
            text = " ".join(whole_text[i].text.split('\n\n'))
    return {'title': title, 
            'text': text}

In [3]:
def get_day_news_links(day):
    links = []
    
    # Формируем url страницы со списком новостей
    url_root = 'http://tramuk.ru/'
    url = url_root + '/date/' + str(day.year) + '/' + str(day.month) + '/' + str(day.day) + '.html'
    
    # получаем текст страницы
    page_text = requests.get(url).text
    bs=BeautifulSoup(page_text, "html5lib")
    
    # ветви дерева с новостями
    whole_text = bs.find_all('div', 'story-list__item')
    for i in range(len(whole_text)):
        # ссылки на новости начинаются с тега h2
        item_title = whole_text[i].find_all("h2")
        for j in range(len(item_title)):
            # Вытаскиваем ссылки в тегах a
            link = item_title[0].find('a').get('href')
            links.append(url_root + link)
    return links

In [4]:
# Сохраняем статьи в файл.
def saveArticle(filename, art, date):
    newsfile=open(filename, 'a', encoding='utf-8')
    title = str(art['title']).encode().decode('utf-8')
    text = str(art['text']).encode().decode('utf-8')
    text = text.strip('\n')
    text = text.strip()
    newsfile.write('\n=====\n'+title)
    newsfile.write('\n' + str(date.year) + '/' + str(date.month) + '/' + str(date.day))
    newsfile.write('\n-----\n'+text)
    newsfile.close()

In [5]:
%%time
filename = 'text2.txt'
newsfile=open(filename, 'w', encoding='utf-8')
newsfile.close()
#start_date = date(2010, 1, 1)
start_date = date(2020, 8, 1)
day = start_date
one_day_delta = timedelta(days=1)
# stop_date = date.today() + one_day_delta
stop_date = date(2020, 8, 4) + one_day_delta
whole_days = stop_date - day
while day != stop_date:
    links_list = []
    links_list = get_day_news_links(day)
    for link in links_list:
        try:
            res = get_news_text(link)
            saveArticle(filename, res, day)
        except Exception as err:
            print(err)
            print('Date: ' + str(day))
            print('step: ' + str((day - start_date).days))
    print(str((day - start_date).days)+"/"+str((whole_days-one_day_delta).days), end='\r')
    
    day += one_day_delta

print(str((day - start_date).days)+"/"+str((whole_days-one_day_delta).days), end='\r')
print('Success!!!\n')

Success!!!

Wall time: 1.97 s
